<a href="https://colab.research.google.com/github/lorenafc/MGI/blob/main/ESTCScrape_chromedrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#source: chatGPT

In [ ]:
!pip install selenium --quiet
!pip install pandas --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.7/467.7 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
pip install --upgrade selenium chromedriver-autoinstaller --quiet

In [ ]:
# !apt-get update
# !apt-get install -y wget unzip
# !pip install selenium
# !wget -N https://chromedriver.storage.googleapis.com/125.0.6422.112/chromedriver_linux64.zip #original chatGPT version 114.0.5735.90
# !unzip -o chromedriver_linux64.zip
# !mv chromedriver /usr/local/bin/chromedriver


In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Set up Selenium with Chrome WebDriver
#used StackOverflow to help adjust https://stackoverflow.com/questions/76461596/unable-to-use-selenium-webdriver-getting-two-exceptions
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Create the WebDriver instance
driver = webdriver.Chrome(options=chrome_options)

# Test if Chrome is working
try:
    driver.get("https://www.google.com/")
    print("Chrome is working")
except Exception as e:
    print(f"Error starting Chrome: {e}")
    driver.quit()
    exit()

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        # Adjust CSS selectors to target elements within the div with id "root"
        title = driver.find_element(By.CSS_SELECTOR, '#root div.title').text
        alt_title = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Alt Title") + td').text
        uniform_title = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Uniform Title") + td').text
        year = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Year") + td').text
        country_name = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Country Name") + td').text
        date = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Date") + td').text
        ustc_classification = driver.find_element(By.CSS_SELECTOR, '#root td:contains("USTC Classification") + td').text
        publication_language = driver.find_element(By.CSS_SELECTOR, '#root td:contains("Publication Language") + td').text
        marc_id = driver.find_element(By.CSS_SELECTOR, '#root td:contains("MARC ID") + td').text

        return {
            'Title': title,
            'Alt Title': alt_title,
            'Uniform Title': uniform_title,
            'Year': year,
            'Country Name': country_name,
            'Date': date,
            'USTC Classification': ustc_classification,
            'Publication Language': publication_language,
            'MARC ID': marc_id
        }
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None



# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}?master_biblio_join_filter_clean_full_holdings%5Brange%5D%5ByearAutoExtracted%5D=%3A{year}"
    driver.get(search_url)

    while True:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
        )
        record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
        record_urls = [element.get_attribute('href') for element in record_elements]

        for record_url in record_urls:
            record_data = scrape_record(record_url)
            if record_data:
                records.append(record_data)

        # Check if there is a next page
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            if 'disabled' in next_button.get_attribute('class'):
                break
            else:
                next_button.click()
                time.sleep(2)
        except:
            break

    return records

# Scrape records year by year
all_records = []
for year in range(1463, 1881):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_books.csv', index=False)
df.to_excel('ESTC_books.xlsx', index=False)

# Close the driver
driver.quit()

print("Scraping completed and data saved to ESTC_books.csv and ESTC_books.xlsx.")


Chrome is working
Scraping year: 1463


TimeoutException: Message: 
Stacktrace:
#0 0x56c6cf484e8a <unknown>
#1 0x56c6cf16e45c <unknown>
#2 0x56c6cf1ba5b5 <unknown>
#3 0x56c6cf1ba671 <unknown>
#4 0x56c6cf1fef14 <unknown>
#5 0x56c6cf1dd4dd <unknown>
#6 0x56c6cf1fc2cc <unknown>
#7 0x56c6cf1dd253 <unknown>
#8 0x56c6cf1ad1c7 <unknown>
#9 0x56c6cf1adb3e <unknown>
#10 0x56c6cf44b2cb <unknown>
#11 0x56c6cf44f377 <unknown>
#12 0x56c6cf437dfe <unknown>
#13 0x56c6cf44fe42 <unknown>
#14 0x56c6cf41c79f <unknown>
#15 0x56c6cf474178 <unknown>
#16 0x56c6cf47434b <unknown>
#17 0x56c6cf483fbc <unknown>
#18 0x7ffac2585ac3 <unknown>


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
# def scrape_record(record_url):
#     driver.get(record_url)
#     try:
#         title = driver.find_element(By.CSS_SELECTOR, 'div.title').text
#         author = driver.find_element(By.CSS_SELECTOR, 'div.author').text
#         year = driver.find_element(By.CSS_SELECTOR, 'div.year').text
#         return {'Title': title, 'Author': author, 'Year': year}
#     except Exception as e:
#         print(f"Error scraping record {record_url}: {e}")
#         return None

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        alt_title = driver.find_element(By.XPATH, '//td[text()="Alt Title"]/following-sibling::td').text
        author = driver.find_element(By.XPATH, '//td[text()="Author"]/following-sibling::td').text
        city = driver.find_element(By.XPATH, '//td[text()="City"]/following-sibling::td').text
        country_code = driver.find_element(By.XPATH, '//td[text()="Country Code"]/following-sibling::td').text
        estc_no = driver.find_element(By.XPATH, '//td[text()="ESTC No"]/following-sibling::td').text
        format = driver.find_element(By.XPATH, '//td[text()="Format"]/following-sibling::td').text
        imprint_original = driver.find_element(By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td').text
        language_code = driver.find_element(By.XPATH, '//td[text()="Language Code"]/following-sibling::td').text
        pagination = driver.find_element(By.XPATH, '//td[text()="Pagination"]/following-sibling::td').text
        title = driver.find_element(By.XPATH, '//td[text()="Title"]/following-sibling::td').text
        year = driver.find_element(By.XPATH, '//td[text()="Year"]/following-sibling::td').text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None


# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}search?q=&year={year}&year_end={year}"
    driver.get(search_url)

    while True:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
            record_urls = [element.get_attribute('href') for element in record_elements]

            for record_url in record_urls:
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            if 'disabled' in next_button.get_attribute('class'):
                break
            else:
                next_button.click()
                time.sleep(2)
        except TimeoutException:
            print("Timed out waiting for records to load. Moving to the next page.")
            break

    return records

### TEST 1473-1475 #####
# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474newfunctionscrape.csv', index=False)
df.to_excel('ESTC_records1473_1474newfunctionscrape.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Timed out waiting for records to load. Moving to the next page.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        attributes = driver.find_elements(By.CSS_SELECTOR, 'table tbody tr')
        record_data = {}
        for attribute in attributes:
            key = attribute.find_element(By.CSS_SELECTOR, 'td:nth-child(1)').text.strip()
            value = attribute.find_element(By.CSS_SELECTOR, 'td:nth-child(2)').text.strip()
            record_data[key] = value
        return record_data
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}search?q=&year={year}&year_end={year}"
    driver.get(search_url)

    while True:
        try:
            WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
            record_urls = [element.get_attribute('href') for element in record_elements]

            for record_url in record_urls:
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            if 'disabled' in next_button.get_attribute('class'):
                break
            else:
                next_button.click()
                time.sleep(2)
        except TimeoutException:
            print("Timed out waiting for records to load. Moving to the next page.")
            break

    return records

# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474_newscrape.csv', index=False)
df.to_excel('ESTC_records1473_1474_newscrape.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Timed out waiting for records to load. Moving to the next page.


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from selenium.common.exceptions import TimeoutException

# Set up Selenium with Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Initialize the Chrome WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Base URL of the website
base_url = "https://estc.printprobability.org/"

# Function to scrape a single record
def scrape_record(record_url):
    driver.get(record_url)
    try:
        alt_title = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Alt Title"]/following-sibling::td'))).text
        author = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Author"]/following-sibling::td'))).text
        city = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="City"]/following-sibling::td'))).text
        country_code = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Country Code"]/following-sibling::td'))).text
        estc_no = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="ESTC No"]/following-sibling::td'))).text
        format = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Format"]/following-sibling::td'))).text
        imprint_original = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Imprint Original"]/following-sibling::td'))).text
        language_code = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Language Code"]/following-sibling::td'))).text
        pagination = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Pagination"]/following-sibling::td'))).text
        title = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Title"]/following-sibling::td'))).text
        year = WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, '//td[text()="Year"]/following-sibling::td'))).text

        return {
            'Alt Title': alt_title,
            'Author': author,
            'City': city,
            'Country Code': country_code,
            'ESTC No': estc_no,
            'Format': format,
            'Imprint Original': imprint_original,
            'Language Code': language_code,
            'Pagination': pagination,
            'Title': title,
            'Year': year
        }
    except Exception as e:
        print(f"Error scraping record {record_url}: {e}")
        return None

# Function to scrape records for a given year
def scrape_year(year):
    records = []
    search_url = f"{base_url}search?q=&year={year}&year_end={year}"
    driver.get(search_url)

    while True:
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div.record"))
            )
            record_elements = driver.find_elements(By.CSS_SELECTOR, "div.record a.view-record")
            record_urls = [element.get_attribute('href') for element in record_elements]

            for record_url in record_urls:
                record_data = scrape_record(record_url)
                if record_data:
                    records.append(record_data)

            # Check if there is a next page
            next_button = driver.find_element(By.CSS_SELECTOR, "a.next")
            if 'disabled' in next_button.get_attribute('class'):
                break
            else:
                next_button.click()
                time.sleep(2)
        except TimeoutException:
            print("Timed out waiting for records to load. Moving to the next page.")
            break

    return records

# Scrape records year by year
all_records = []
for year in range(1473, 1475):
    print(f"Scraping year: {year}")
    records = scrape_year(year)
    all_records.extend(records)

# Save to CSV and Excel
df = pd.DataFrame(all_records)
df.to_csv('ESTC_records1473_1474.csv', index=False)
df.to_excel('ESTC_records1473_1474.xlsx', index=False)

# Close the driver
driver.quit()


Scraping year: 1473
Timed out waiting for records to load. Moving to the next page.
Scraping year: 1474
Timed out waiting for records to load. Moving to the next page.
